## Importation des données

In [5]:
import sys
from pathlib import Path
import importlib
import ipynbname 
import pandas as pd
import geopandas as gpd
from datetime import datetime

code_path = ipynbname.path().parent.parent
# Ajouter le dossier scripts au path
scripts_path = code_path  / "scripts"
base_path=code_path.parent
sys.path.append(str(scripts_path.resolve()))

import data_utils  # importe le module une première fois

# Après avoir modifié data_utils.py
importlib.reload(data_utils)

# Maintenant tu peux accéder aux fonctions mises à jour
from data_utils import import_data_raw, import_data_sig, melt_long_format, clean_year_column, save_long_dataframe, concat_intermediate_files


In [6]:
#Country data
filename="data_final_all_norm.csv"
filepath= base_path/ "Data" / 'data_final' / filename

df_data = pd.read_csv(filepath)
df_data.head(100)

,Year,Country,Value,Unit,Indicator,Source,Country_code,country,gdp_percapita_current_usd,gdp_percapita_ppp_current_intl,...,Value_norm_gdp,Unit_norm_gdp,Value_norm_ppp,Unit_norm_ppp,Value_norm_gdp_hab,Unit_norm_gdp_hab,Value_norm_ppp_hab,Unit_norm_ppp_hab,Value_norm_densite,Unit_norm_densite
0,1960,Afghanistan,4.17891,TgC/year,LULUCF Net emissions,GCB,AF,Afghanistan,NaN,NaN,...,NaN,TgC/year/billion USD,NaN,TgC/year/billion PPP$,NaN,TgC/year/thousand USD/hab,NaN,TgC/year/thousand PPP$/hab,NaN,TgC/year/hab/km²
1,1961,Afghanistan,3.59662,TgC/year,LULUCF Net emissions,GCB,AF,Afghanistan,NaN,NaN,...,NaN,TgC/year/billion USD,NaN,TgC/year/billion PPP$,NaN,TgC/year/thousand USD/hab,NaN,TgC/year/thousand PPP$/hab,0.254591,TgC/year/hab/km²
2,1962,Afghanistan,3.32416,TgC/year,LULUCF Net emissions,GCB,AF,Afghanistan,NaN,NaN,...,NaN,TgC/year/billion USD,NaN,TgC/year/billion PPP$,NaN,TgC/year/thousand USD/hab,NaN,TgC/year/thousand PPP$/hab,0.230543,TgC/year/hab/km²
3,1963,Afghanistan,3.23023,TgC/year,LULUCF Net emissions,GCB,AF,Afghanistan,NaN,NaN,...,NaN,TgC/year/billion USD,NaN,TgC/year/billion PPP$,NaN,TgC/year/thousand USD/hab,NaN,TgC/year/thousand PPP$/hab,0.219361,TgC/year/hab/km²
4,1964,Afghanistan,2.79609,TgC/year,LULUCF Net emissions,GCB,AF,Afghanistan,NaN,NaN,...,NaN,TgC/year/billion USD,NaN,TgC/year/billion PPP$,NaN,TgC/year/thousand USD/hab,NaN,TgC/year/thousand PPP$/hab,0.185820,TgC/year/hab/km²
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1991,Albania,0.07904,TgC/year,LULUCF Net emissions,GCB,AL,Albania,336.586995,1988.181406,...,0.071883,TgC/year/billion USD,0.012169,TgC/year/billion PPP$,0.234828,TgC/year/thousand USD/hab,0.039755,TgC/year/thousand PPP$/hab,0.000663,TgC/year/hab/km²
96,1992,Albania,0.04298,TgC/year,LULUCF Net emissions,GCB,AL,Albania,200.852220,1898.820304,...,0.065903,TgC/year/billion USD,0.006971,TgC/year/billion PPP$,0.213988,TgC/year/thousand USD/hab,0.022635,TgC/year/thousand PPP$/hab,0.000363,TgC/year/hab/km²
97,1993,Albania,-0.06407,TgC/year,LULUCF Net emissions,GCB,AL,Albania,367.279225,2142.681185,...,-0.054053,TgC/year/billion USD,-0.009265,TgC/year/billion PPP$,-0.174445,TgC/year/thousand USD/hab,-0.029902,TgC/year/thousand PPP$/hab,-0.000544,TgC/year/hab/km²
98,1994,Albania,-0.17971,TgC/year,LULUCF Net emissions,GCB,AL,Albania,586.416135,2384.728785,...,-0.095542,TgC/year/billion USD,-0.023494,TgC/year/billion PPP$,-0.306455,TgC/year/thousand USD/hab,-0.075359,TgC/year/thousand PPP$/hab,-0.001535,TgC/year/hab/km²


In [7]:
#Global sig
gdf_world=import_data_sig('world.geojson',base_path)


def simplify_geom(geom, tol=0.1):
    if geom is None:
        return None
    if geom.geom_type == 'Polygon':
        return geom.simplify(tol, preserve_topology=True)
    elif geom.geom_type == 'MultiPolygon':
        return type(geom)([poly.simplify(tol, preserve_topology=True) for poly in geom.geoms])
    return geom
    
gdf_world = gdf_world[gdf_world['Country_code'].notna()].copy()
gdf_world['geometry'] = gdf_world['geometry'].apply(lambda g: simplify_geom(g, tol=0.1))
gdf_world.head(10)


,geo_point_2d,iso3,status,color_code,name,continent,region,Country_code,french_short,Country,geometry
2,"{ ""lon"": 9.5613358449883421, ""lat"": 34.1108585...",TUN,Member State,TUN,Tunisia,Africa,Northern Africa,TN,Tunisie,Tunisia,"MULTIPOLYGON (((10.96833 33.8575, 10.865 33.63..."
3,"{ ""lon"": 43.77213543247138, ""lat"": 33.04802449...",IRQ,Member State,IRQ,Iraq,Asia,Western Asia,IQ,Iraq,Iraq,"POLYGON ((44.78734 37.14971, 44.85304 36.79458..."
4,"{ ""lon"": -6.3178452255610269, ""lat"": 31.883624...",MAR,Member State,MAR,Morocco,Africa,Northern Africa,MA,Maroc,Morocco,"POLYGON ((-2.94694 35.32916, -2.87694 35.1418,..."
5,"{ ""lon"": 25.088778010977098, ""lat"": -28.993203...",ZAF,Member State,ZAF,South Africa,Africa,Southern Africa,ZA,Afrique du Sud,South Africa,"POLYGON ((31.2975 -22.41476, 31.55083 -23.4766..."
6,"{ ""lon"": -61.253176819842295, ""lat"": 10.468641...",TTO,Member State,TTO,Trinidad and Tobago,Americas,Caribbean,TT,Trinité-et-Tobago,Trinidad and Tobago,"MULTIPOLYGON (((-60.92306 10.79722, -61.00445 ..."
7,"{ ""lon"": 2.5519552167777979, ""lat"": 46.5645020...",FRA,Member State,FRA,France,Europe,Western Europe,FR,France,France,"MULTIPOLYGON (((9.45958 42.98805, 9.55257 42.1..."
9,"{ ""lon"": 168.62692074653282, ""lat"": 7.64327257...",MHL,Member State,MHL,Marshall Islands,Oceania,Micronesia,MH,Îles Marshall,Marshall Islands,"MULTIPOLYGON (((168.13098 5.62236, 168.09497 5..."
10,"{ ""lon"": 12.071742970356828, ""lat"": 42.7957823...",ITA,Member State,ITA,Italy,Europe,Southern Europe,IT,Italie,Italy,"MULTIPOLYGON (((15.64794 38.26457, 15.0925 37...."
11,"{ ""lon"": -53.089819954013173, ""lat"": -10.77310...",BRA,Member State,BRA,Brazil,Americas,South America,BR,Brésil,Brazil,"MULTIPOLYGON (((-48.55056 -27.82139, -48.51778..."
12,"{ ""lon"": 29.871837767180068, ""lat"": -19.000098...",ZWE,Member State,ZWE,Zimbabwe,Africa,Eastern Africa,ZW,Zimbabwe,Zimbabwe,"POLYGON ((30.41576 -15.63187, 30.42236 -16.005..."


In [139]:
import webbrowser

# Nom du fichier HTML de sortie
output_file = "map_LULUCF_slider_light"

# Appel de la fonction avec slider pour l'année
plot_world_map_with_slider(
    df_data[df_data['Year'].isin([1980,1990,2000,2010,2020])],
    gdf_world=gdf_world,
    indicator="LULUCF Net emissions",
    output_html=output_file,
    save=True,
    base_path=base_path
)


Carte interactive sauvegardée dans C:\Users\Aubin\Documents\NetZero\Figures\map_LULUCF_slider_light_20251112.html


In [17]:
df_data

,countryiso3,Year,country,gdp_percapita_current_usd,gdp_percapita_ppp_current_intl,missing_population_total,missing_gdp_current_usd,missing_gdp_percapita_current_usd,missing_gdp_ppp_current_intl,missing_gdp_percapita_ppp_current_intl,...,Value_norm_gdp,Unit_norm_gdp,Value_norm_ppp,Unit_norm_ppp,Value_norm_gdp_hab,Unit_norm_gdp_hab,Value_norm_ppp_hab,Unit_norm_ppp_hab,Value_norm_densite,Unit_norm_densite
0,1A,1960,Arab World,NaN,NaN,False,True,True,True,True,...,NaN,/billion USD,NaN,/billion PPP$,NaN,/thousand USD/hab,NaN,/thousand PPP$/hab,NaN,/hab/km²
1,1A,1961,Arab World,212.889663,NaN,False,False,False,True,True,...,19.997084,/billion USD,NaN,/billion PPP$,0.212890,/thousand USD/hab,NaN,/thousand PPP$/hab,8.360295,/hab/km²
2,1A,1962,Arab World,210.805415,NaN,False,False,False,True,True,...,20.327671,/billion USD,NaN,/billion PPP$,0.210805,/thousand USD/hab,NaN,/thousand PPP$/hab,8.582530,/hab/km²
3,1A,1963,Arab World,225.800630,NaN,False,False,False,True,True,...,22.362958,/billion USD,NaN,/billion PPP$,0.225801,/thousand USD/hab,NaN,/thousand PPP$/hab,8.814823,/hab/km²
4,1A,1964,Arab World,243.897432,NaN,False,False,False,True,True,...,24.811627,/billion USD,NaN,/billion PPP$,0.243897,/thousand USD/hab,NaN,/thousand PPP$/hab,9.054355,/hab/km²
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17285,ZW,2020,Zimbabwe,1730.453910,3510.676040,False,False,False,False,False,...,26.868564,/billion USD,54.509874,/billion PPP$,1.730454,/thousand USD/hab,3.510676,/thousand PPP$/hab,40.136714,/hab/km²
17286,ZW,2021,Zimbabwe,1724.387271,3184.784602,False,False,False,False,False,...,27.240508,/billion USD,50.310711,/billion PPP$,1.724387,/thousand USD/hab,3.184785,/thousand PPP$/hab,40.835492,/hab/km²
17287,ZW,2022,Zimbabwe,2040.546587,3560.039403,False,False,False,False,False,...,32.789657,/billion USD,57.206473,/billion PPP$,2.040547,/thousand USD/hab,3.560039,/thousand PPP$/hab,41.538209,/hab/km²
17288,ZW,2023,Zimbabwe,2156.034093,3820.359922,False,False,False,False,False,...,35.231369,/billion USD,62.427821,/billion PPP$,2.156034,/thousand USD/hab,3.820360,/thousand PPP$/hab,42.240719,/hab/km²


## with dash

In [168]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import pandas as pd
import geopandas as gpd
import json
import numpy as np

# --- Fonction symlog ---
def symlog(x):
    """Logarithme symétrique : log10(|x|+1) * signe(x)"""
    if pd.isna(x):
        return None
    return np.sign(x) * np.log10(abs(x) + 1)

# --- Normalisations ---
norm_map = {
    'No norm': ('Value', 'Unit'),
    'Area': ('Value_norm_area', 'Unit_norm_area'),
    'Population': ('Value_norm_population', 'Unit_norm_population'),
    'Hab/km2': ('Value_norm_densite', 'Unit_norm_densite'),
    'PPP': ('Value_norm_ppp', 'Unit_norm_ppp'),
    'GDP': ('Value_norm_gdp', 'Unit_norm_gdp'),
    'PPP/hab': ('Value_norm_ppp_hab', 'Unit_norm_ppp_hab'),
    'GDP/hab': ('Value_norm_gdp_hab', 'Unit_norm_gdp_hab')
}

# --- Exemple de structure des données (à remplacer par tes vraies données) ---
# df_data = pd.read_csv("ton_fichier.csv")
# gdf_world = gpd.read_file("ton_shapefile.geojson")

# ⚠️ Correction : s’assurer que Year est bien de type int
df_data['Year'] = df_data['Year'].astype(int)

# --- Initialiser Dash ---
app = JupyterDash(__name__)
app.title = "Carte interactive mondiale"

# --- CSS plein écran ---
app.index_string = '''
<!DOCTYPE html>
<html>
    <head>
        {%metas%}
        <title>{%title%}</title>
        {%favicon%}
        {%css%}
        <style>
            body, html {
                margin:0;
                padding:0;
                width:100%;
                height:100%;
                overflow: hidden;
            }
        </style>
    </head>
    <body>
        {%app_entry%}
        <footer style="display:none;">
            {%config%}
            {%scripts%}
            {%renderer%}
        </footer>
    </body>
</html>
'''

# --- Layout Dash ---
app.layout = html.Div([
    html.Div([
        html.H1("Carte mondiale interactive", style={'marginBottom': '10px'}),

        html.Div([
            html.Label("Choisir l'indicateur :"),
            dcc.Dropdown(
                id='dropdown-indicator',
                options=[{'label': ind, 'value': ind} for ind in sorted(df_data['Indicator'].unique())],
                value=df_data['Indicator'].unique()[0]
            )
        ], style={'width': '32%', 'display': 'inline-block', 'marginRight': '1%'}),

        html.Div([
            html.Label("Choisir normalisation :"),
            dcc.Dropdown(
                id='dropdown-norm',
                options=[{'label': n, 'value': n} for n in norm_map.keys()],
                value='No norm'
            )
        ], style={'width': '32%', 'display': 'inline-block', 'marginRight': '1%'}),

        html.Div([
            html.Label("Choisir échelle :"),
            dcc.Dropdown(
                id='dropdown-scale',
                options=[{'label': s, 'value': s} for s in ['absolute', 'relative', 'rank', 'log']],
                value='relative'
            )
        ], style={'width': '32%', 'display': 'inline-block'}),
    ], style={'padding': '10px', 'backgroundColor': '#f5f5f5'}),

    html.Div([
        html.Label("Année :", style={'fontWeight': 'bold', 'marginTop': '10px'}),
        dcc.Slider(
            id='slider-year',
            min=df_data['Year'].min(),
            max=df_data['Year'].max(),
            step=1,
            value=df_data['Year'].min(),
            marks={int(y): str(y) for y in sorted(df_data['Year'].unique())},
            tooltip={"placement": "bottom", "always_visible": False}
        )
    ], style={'padding': '15px 30px 0px 30px'}),

    dcc.Graph(id='world-map', style={'width': '100%', 'height': 'calc(100vh - 230px)', 'marginBottom': '10px'})
])

# --- Callback ---
@app.callback(
    Output('world-map', 'figure'),
    Input('dropdown-indicator', 'value'),
    Input('slider-year', 'value'),
    Input('dropdown-norm', 'value'),
    Input('dropdown-scale', 'value')
)
def update_map(indicator, year, norm_name, scale):
    # ✅ Sécurité : convertir 'year' en int pour le filtrage
    year = int(year)

    # Filtrer les données
    df_filtered = df_data[(df_data['Indicator'] == indicator) & (df_data['Year'] == year)].copy()

    if df_filtered.empty:
        return go.Figure().update_layout(
            title=f"Aucune donnée pour {indicator} en {year}",
            paper_bgcolor='white'
        )

    gdf_merged = gdf_world.merge(df_filtered, on='Country_code', how='left')
    geojson_data = json.loads(gdf_merged.to_json())

    # Colonnes de normalisation
    col_value, col_unit = norm_map[norm_name]
    z_plot = gdf_merged[col_value]
    unit_val = gdf_merged[col_unit].iloc[0] if col_unit in gdf_merged.columns else ""

    # Transformation selon échelle
    if scale == 'rank':
        z_plot = z_plot.rank(ascending=True)
    elif scale == 'log':
        z_plot = z_plot.apply(symlog)
    elif scale == 'absolute':
        z_plot = z_plot.abs()

    # Création de la figure
    fig = go.Figure(go.Choropleth(
        geojson=geojson_data,
        locations=gdf_merged.index,
        z=z_plot,
        text=gdf_merged['name'],
        colorscale='RdYlGn_r',
        colorbar_title=f"{indicator} ({unit_val})",
        hovertemplate=f"<b>%{{text}}</b><br>{indicator} ({norm_name}) = %{{z:.2f}} {unit_val}<extra></extra>"
    ))

    fig.update_layout(
        geo=dict(
            showcountries=True,
            showcoastlines=True,
            showland=True,
            landcolor="lightgray",
            oceancolor="lightblue",
            projection_type="natural earth",
            fitbounds="locations"
        ),
        margin=dict(l=0, r=0, t=0, b=0),
        paper_bgcolor='white',
        plot_bgcolor='white'
    )

    return fig


C:\Users\Aubin\AppData\Local\Programs\Python\Python313\Lib\site-packages\dash\dash.py:634: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



In [169]:
# --- Lancer ---
if __name__ == "__main__":
    import webbrowser
    url = "http://127.0.0.1:8050/"
    webbrowser.open_new_tab(url)
    app.run(debug=True, port=8050, threaded=True)